In [65]:
# Импортируем библиотеки pandas и numpy
# Смотрим общую информацию
import pandas as pd
import numpy as np
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [66]:
# Посмотрим на саму таблицу (первые 15 строк)
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Таблица состоит из ***12 столбцов и 21525 строк***. Особое внимание обратим на столбцы *'days_employed'* и *'total_income'* : формат данных в этих стобацих вещественный, в то время как для обработки данных нам потребуется целочисленный. Так же только в этих двух столбцах есть пропущенных значения (***всего заполнено 19351 строк из 21525***).

В столбце *'days_employed'* так же ***присутствуют отрицательные значения***, что говорит об ошибке (т.к. в этом столбце указан стаж работы, который не может быть отрицательным числом). Столбец *'education'* так же потребует дополнительной обработки, т.к. ***текст там написан с разным регистром*** и при группировке данных возможна ошибка. 

### Шаг 2. Предобработка данных

### Обработка пропусков

In [67]:
# Для заполненения пустых строк в столбце 'days_employed' используем медианное значение.
# Для начала удалим все строки без данных ("NaN"), результат сохраним в переменную data_dropped_employed
data_dropped_employed = data.dropna()
data_dropped_employed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21524
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null float64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null float64
purpose             19351 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 1.9+ MB


In [68]:
# Посмотрим минимальное значение по столбцу 'days_employed'
data_dropped_employed.days_employed.min()

-18388.949900568383

In [69]:
# Как можно заметить - в данном столбце у нас ошибка. Вероятная причина ошибки в следующем: стаж работы в днях высчитывается
# путем вычитания из даты окончания работы дату начала работы (time_end - time_start) для каждого рабочего места.
# После чего стажи со всех рабочих мест - суммируются. Вероятно, ошибка возникла в следствии того, что при подсчете дата
# начала работы и дата окончания работы были перепутаны местами (time_start - time_end). 
# Решить данную ошибку поможет простой перевод из отрицательного числа в положительное 
# (для этого достаточно использовать функцию abs()).
data_dropped_employed['days_employed'] = data_dropped_employed['days_employed'].abs()
data_dropped_employed.head(15)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [70]:
# И опять мы видем некорректные значения. Количество стажа в днях у пенсионеров очень завышено. 
# Возможная причина этого - стаж указан не в днях, а в часах. Чтобы получить корректное значение разделим данные на 24.
data_dropped_employed.loc[data_dropped_employed['income_type'] == 'пенсионер', 'days_employed'] /= 24
data_dropped_employed['days_employed'] = data_dropped_employed['days_employed'].astype('int64')
data_dropped_employed.head(15)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [71]:
# Вычеслим медиану
data_dropped_employed['days_employed'].median()

2194.0

In [72]:
#Теперь нам известна медиана, которую мы можем вставить в исходную таблицу вместо пропусков
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [73]:
# По аналогии с data_dropped убираем отрицательные значения и редактируем стаж работы для пенсионеров
data['days_employed'] = data['days_employed'].abs()
data.loc[data['income_type'] == 'пенсионер', 'days_employed'] /= 24
data['days_employed'] = data['days_employed'].fillna(2194)
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [74]:
# Теперь заполним пропуска в столбце total_income. Для этого нам понадобится удалить все недостающий значения, узнать медиану
# и вставить её вместо пропусков
data_income = data.dropna()
data_income['total_income'].median()

145017.93753253992

In [75]:
# Вставляем медианное значение вместо пропусков.
data['total_income'] = data['total_income'].fillna(145017.93753253992)
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<font color='brown'>Однако можно пойти немного дальше и попробовать заполнить пропуски медианами по роду деятельности: пенсионеров - медианами пенсионеров, сотрудников - медианами сотрудников. Так заполнение пропусков будет еще достовернее.

### Вывод

В таблице ***частично отсутствовали значения*** в столбцах *'days_employed'* и *'total_income'*. В связи с тем, что отсутствующие значения составляют около 10% от всех данных - просто избавиться от них путем удаления считаю важной потерей для анализа. В связи с этим в столбцах были вычеслены медианы, которые и использовались для замены пустых значений. 

Так же в столбце *'days_employed'* ***были некорректные данные (такие, как отрицательный стаж или слишком огромные стаж работы)***, которые были исправлены путем переноса значения с отрицательного на положительное, а так же переводом часов в дни

### Замена типа данных

In [76]:
# На данном этапе сведем все вещественные типы данных на целочисленные
data['days_employed']=data['days_employed'].astype('int64')
data['total_income']=data['total_income'].astype('int64')
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Вывод

Для изменения типа данных использовалась функция *.astype()*, после использования которой все столбцы с количественными переменными приобрели одинаковый формат *(int64)*. База данных приобрела куда более удобный и понятный вид, подходящий для дальнейшей обработки.

### Обработка дубликатов

In [77]:
# Для начала проверим количество дубликатов
data.duplicated().sum()

54

In [78]:
# Проверим список всех значений в столбце 'education'
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [79]:
#Так же проверим список всех значений в столбце 'gender'
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [80]:
# Проверим список всех значений в столбце'childer'
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [81]:
# Как было известно ранее - в столбце 'education' используется разный регистр, из-за чего часть дубликатов могла быть упущена.
data['education'] = data['education'].str.lower()
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [82]:
# Так же в строке 'gender' было найдено пропущенное значение. Т.к. восстановить данные самостоятельно нет возможности, а строка
# всегда одна - для более корректных данных строку можно удалить.
data = data.loc[data['gender'] != 'XNA']
data['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

In [83]:
# Скорее всего отрицательное количество детей - опечатка, поэтому переведем значения из отрицательных в положительные
data['children']=data['children'].abs()
data['children'].value_counts()

0     14148
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [84]:
# Детей в количестве 20 шт. можно увидеть в 76 случаях, что маловероятно соответствует действительности 
# (скорее всего ошибка оператора). Для корректности анализа данные строки будут удалены.
data = data.loc[data['children'] != 20]
data['children'].value_counts()

0    14148
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [85]:
# Проверим количество дубликатов после обработки
data.duplicated().sum()

71

In [86]:
# Количество дубликатов увеличилось, а значит регистр непосредственно влияет на результат. Теперь можем удалить дубликаты.
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

### Вывод

Прежде чем начать обрабатывать дубликаты - для начала нужно обработать все столбцы, проверить на наличие уникальных данных и одного формата написания (для этого можно использовать *.value_counts()* и *.unique()*). После обработки всей таблицы ***количество дубликатов было увеличено (71 вместо 54)***. Для удаления дубликатов использовалась функция *.drop_duplicates()* 

Скорее всего дубликаты и уникальные данные ***возникли по причине ошибки оператора, либо из-за повторного ввода данных (например, задублированный запрос от клиента)***. Имея чуть больше данных - можно значения обрабатывать иначе (например, если бы было известно имя человека - можно было бы исправить столбец 'пол').

### Лемматизация

In [87]:
# Импортируем библиотеку с функцией лемматизации. 
# Для понимания общей картины - посчитаем количества всех данных в столбце purpose
from pymystem3 import Mystem
m = Mystem()
data.purpose.value_counts()

свадьба                                   790
на проведение свадьбы                     764
сыграть свадьбу                           761
операции с недвижимостью                  674
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   650
операции с жильем                         648
операции с коммерческой недвижимостью     646
покупка жилья                             643
жилье                                     642
покупка жилья для семьи                   637
недвижимость                              632
строительство собственной недвижимости    629
операции со своей недвижимостью           626
строительство жилой недвижимости          623
покупка своего жилья                      620
строительство недвижимости                619
покупка недвижимости                      618
ремонт жилью                              605
покупка жилой недвижимости                604
на покупку своего автомобиля              505
заняться высшим образованием      

In [88]:
# Как можно увидеть - есть похожие значения с разным описанием (например, 'покупка жилья' и 'операции с жильем'). 
# При анализе эти данные будут групироваться в разных строчках, что может вызвать некорректный анализ. 
# Наша задача привести данные с одним смыслом к одному формату. 
# Создаем отдельную базу данных из столбца 'purpose' и лемматизируем её. Для проверки - печатаем первые 15 результатов
general_purpose = data.purpose
general_purpose = general_purpose.apply(m.lemmatize)
general_purpose.head(15)

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
5                             [покупка,  , жилье, \n]
6                      [операция,  , с,  , жилье, \n]
7                                   [образование, \n]
8                 [на,  , проведение,  , свадьба, \n]
9           [покупка,  , жилье,  , для,  , семья, \n]
10                     [покупка,  , недвижимость, \n]
11    [покупка,  , коммерческий,  , недвижимость, \n]
12                          [сыграть,  , свадьба, \n]
13                  [приобретение,  , автомобиль, \n]
14           [покупка,  , жилой,  , недвижимость, \n]
Name: purpose, dtype: object

In [89]:
# Теперь мы можем добавить лемматизированные данные в таблицу. 
# Чтобы не терять исходные данные - создаем столбец с названием 'lemmas'
data['lemmas'] = general_purpose
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]"
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]"
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]"
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]"
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]"


In [90]:
data_join_lemmas = ' '.join(data['purpose'])
data_join_lemmas = m.lemmatize(data_join_lemmas)
from collections import Counter
Counter(data_join_lemmas).most_common(10)

[(' ', 54821),
 ('недвижимость', 6329),
 ('покупка', 5877),
 ('жилье', 4445),
 ('автомобиль', 4290),
 ('образование', 3998),
 ('с', 2905),
 ('операция', 2594),
 ('свадьба', 2315),
 ('свой', 2226)]

In [91]:
# После лемматизации стало заметно, что основных целей получения кредита всего 5. 
# Для последующего удобства группировки приведем значения в столбце к одному типу
# Создадим функцию 'one_type', которая будет искать в столбце с уже лемматизированными данными определенный ключ.
# Данная функция так же будет возвращать нам ключевое значение в столбец, чтобы потом можно было по ним группировать.
def one_type(data):
    for row in data:
        purpose = data['lemmas']
        
        if 'автомобиль' in purpose:
            return 'автомобиль'
        elif 'жилье' in purpose:
            return 'жилье'
        elif 'недвижимость' in purpose:
            return 'недвижимость'
        elif 'образование' in purpose:
            return 'образование'
        elif 'свадьба' in purpose:
            return 'свадьба'
        else:
            return 'прочее'
data['lemmas'] = data.apply(one_type, axis = 1)
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье


### Вывод

Для лемматизации мы использовали библиотеку *pymystem3*. Были категоризированны данные в столбце *'lemmas'*. Теперь таблица получила более удобный формат, который позволит нам проводить анализ по целям кредита. Для этого была создана специальная ***функция, проверяющая значения по ключу и возвращающая ключ обратно в столбец.***

### Категоризация данных

In [92]:
# Создадим 2 словаря по характеристикам образование и семейное положение (education_id и family_status_id).
# Начнем со столбца edication_id
education_dict = data[['education_id', 'education']]
education_dict.head(15)

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее
5,0,высшее
6,0,высшее
7,1,среднее
8,0,высшее
9,1,среднее


In [93]:
# Удалим все дубликаты и отсортируем по столбцу education_id
education_dict = education_dict.drop_duplicates().reset_index(drop=True).sort_values('education_id', ascending=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [94]:
# Теперь у нас появился словарь для столбца education_id. Аналогично сделаем словарь для столбца family_status_id
family_status_dict = data[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True).sort_values('family_status_id', ascending=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [95]:
# Теперь уберем из таблицы столбцы education и family_status.

data = data.drop(['education', 'family_status'], axis=1)

data.head(15)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,жилье
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,жилье
7,0,152,50,1,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье


### Вывод

Было создано ***2 словаря*** по столбцам *family_statys_id* и *education_id*. Из списка мы убрали необязательные строки. Список приобрел более упрощенный вариант, который легче поддается анализу. 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [96]:
# Сгруппируем таблицу по столбцу children, при группировке посчитаем общее количество взявших кредит и количество должников.
data_children_grouped = data.groupby('children').agg({'debt': ['sum', 'count']})
data_children_grouped

debt       
           sum  count
children             
0         1063  14090
1          445   4855
2          194   2052
3           27    330
4            4     41
5            0      9

In [97]:
# Посчитаем отношение должников к общему числу представителей группы
data_children_grouped['children_debt'] = data_children_grouped['debt']['sum'] / data_children_grouped['debt']['count']
data_children_grouped

debt        children_debt
           sum  count              
children                           
0         1063  14090      0.075444
1          445   4855      0.091658
2          194   2052      0.094542
3           27    330      0.081818
4            4     41      0.097561
5            0      9      0.000000

## Вывод

Лучше всего, согласно статистике, кредиты отдают люди не имеющие детей (около 7,5%), либо имеющией 5 детей (0%, но данных слишком мало для точного анализа). Хуже всего отдают кредиты семьи с 4, 2 и 1 детьми (9,7%, 9,4% и 9,2% соответсвенно).


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [141]:
# По аналогии с предыдущим анализом группируем по столбцу family_status_id
data_family_grouped = data.groupby('family_status_id').agg({'debt': ['sum', 'count']})
data_family_grouped['family_debt'] = data_family_grouped['debt']['sum']/data_family_grouped['debt']['count']
data_family_grouped

debt        family_debt
                  sum  count            
family_status_id                        
0                 928  12290    0.075509
1                 385   4138    0.093040
2                  63    955    0.065969
3                  84   1193    0.070411
4                 273   2801    0.097465

In [142]:
# Для удобства вернем словарь по столбцу family_status_id
data_family_grouped.merge(family_status_dict, on='family_status_id', how='right')

,family_status_id,"(debt, sum)","(debt, count)","(family_debt, )",family_status
0,0,928,12290,0.075509,женат / замужем
1,1,385,4138,0.093040,гражданский брак
2,2,63,955,0.065969,вдовец / вдова
3,3,84,1193,0.070411,в разводе
4,4,273,2801,0.097465,Не женат / не замужем


In [138]:
family_pivot = data.pivot_table(index='family_status_id', values='debt', aggfunc=['sum', 'count']) # Создаем список
# Высчитываем отношение должников к общему количеству участников группы
family_pivot['family_debt'] = family_pivot['sum']['debt']/family_pivot['count']['debt']
# Для удобства добавляем столбец family_status из словаря family_status_dict
family_pivot = family_pivot.merge(family_status_dict, on='family_status_id', how='right')
family_pivot

,family_status_id,"(sum, debt)","(count, debt)","(family_debt, )",family_status
0,0,928,12290,0.075509,женат / замужем
1,1,385,4138,0.093040,гражданский брак
2,2,63,955,0.065969,вдовец / вдова
3,3,84,1193,0.070411,в разводе
4,4,273,2801,0.097465,Не женат / не замужем


***Итак*** один и тот же результат мы достигли двумя способами, при помощи метода *.groupby()* и метода *.pivot_table()*. Данные и визуальная часть полностью идентичны, а значит, в данном случае, ***оба способа подходят нам одинаково***.


### Вывод

Самый высокий процент задолженности у групп "Не женат/не замужем" и "Гражданский брак" - более 9%. Меньше всего задолженностей у группы "вдовец / вдова", около 6,5%.

<font color='blue'>В целом, здесь видно, что наиболее надежные заемщики, это те, кто находится или когда-либо был в браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [239]:
# Столбец total_income в данном случае неудобен для анализа, нет возможности группировать данные. 
# Для удобства создадим новый столбец total_income_grouped с 5 уровнями дохода
# Создадим для этого функцию
def grouped_income(data):
    for row in data:
        income = data['total_income']
        
        if income <= 100000:
            return 'до 1000000'
        elif income <= 150000:
            return 'от 100001 до 150000'
        elif income <= 200000:
            return 'от 150001 до 200000'
        elif income <= 250000:
            return 'от 200000 до 250000'
        else:
            return 'более 250000'
data['total_income_grouped'] = data.apply(grouped_income, axis = 1)
data.head(15)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,total_income_grouped,total_debt
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,жилье,более 250000,0.091658
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,от 100001 до 150000,0.091658
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,жилье,от 100001 до 150000,0.075444
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,образование,более 250000,0.081818
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,от 150001 до 200000,0.075444
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,жилье,более 250000,0.075444
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,жилье,от 200000 до 250000,0.075444
7,0,152,50,1,0,M,сотрудник,0,135823,образование,образование,от 100001 до 150000,0.075444
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,до 1000000,0.094542
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье,от 100001 до 150000,0.075444


In [240]:
# Аналогично предыдущим анализам высчитываем отношение числа участников группы на число задолженностей.
data_income_grouped = data.groupby('total_income_grouped').agg({'debt':['sum', 'count']})
data_income_grouped['income_debt'] = data_income_grouped['debt']['sum']/data_income_grouped['debt']['count']
data_income_grouped

debt       income_debt
                      sum count            
total_income_grouped                       
более 250000          194  2801    0.069261
до 1000000            354  4452    0.079515
от 100001 до 150000   657  7772    0.084534
от 150001 до 200000   366  4108    0.089094
от 200000 до 250000   162  2244    0.072193

In [241]:
# Создаем список
income_table = data.pivot_table(index='total_income_grouped', values='debt', aggfunc=['sum', 'count'])
income_table

,sum,count
,debt,debt
total_income_grouped,,
более 250000,194,2801
до 1000000,354,4452
от 100001 до 150000,657,7772
от 150001 до 200000,366,4108
от 200000 до 250000,162,2244


In [242]:
# Высчитываем отношение
income_table['income_debt'] = income_table['sum']['debt']/income_table['count']['debt']
income_table

,sum,count,income_debt
,debt,debt,
total_income_grouped,,,
более 250000,194,2801,0.069261
до 1000000,354,4452,0.079515
от 100001 до 150000,657,7772,0.084534
от 150001 до 200000,366,4108,0.089094
от 200000 до 250000,162,2244,0.072193


### Вывод

Люди с самым высоким уровнем дохода имеют меньший процент задолженности (чуть меньше 7%). Самые большие категории - доход от 100001 до 150000 и доход от 150001 до 200000 (8,5% и 9% соответственно).

<font color='blue'>Также можно заметить, что клиенты с самым низким уровнем дохода на удивление неплохо справляются с выплатами.

- Как разные цели кредита влияют на его возврат в срок?

In [37]:
# Проделываем работу аналогичную предыдущим вариантам
data_purpose_grouped = data.groupby('lemmas').agg({'debt':['sum', 'count']})
data_purpose_grouped['purpose_debt'] = data_purpose_grouped['debt']['sum']/data_purpose_grouped['debt']['count']
data_purpose_grouped

debt       purpose_debt
              sum count             
lemmas                              
автомобиль    401  4290     0.093473
жилье         308  4445     0.069291
недвижимость  472  6329     0.074577
образование   369  3998     0.092296
свадьба       183  2315     0.079050

In [156]:
# Все делаем аналогично предыдущим заданиям
purpose_table = data.pivot_table(index='lemmas', values='debt', aggfunc=['sum', 'count'])
purpose_table['purpose_debt']=purpose_table['sum']['debt']/purpose_table['count']['debt']
purpose_table

,sum,count,purpose_debt
,debt,debt,
lemmas,,,
автомобиль,401,4290,0.093473
жилье,308,4445,0.069291
недвижимость,472,6329,0.074577
образование,369,3998,0.092296
свадьба,183,2315,0.079050


### Попробуем посмотреть статистику людей, берущих кредит на свадьбу

In [181]:
data_try = data.loc[data['lemmas'] == 'свадьба']
data_try

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,total_income_grouped
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,от 150001 до 200000
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,до 1000000
12,0,2194,65,1,1,M,пенсионер,0,145017,сыграть свадьбу,свадьба,от 1000001 до 150000
23,0,272,21,0,1,M,сотрудник,0,128265,сыграть свадьбу,свадьба,от 1000001 до 150000
32,0,4649,34,1,1,F,сотрудник,1,139057,на проведение свадьбы,свадьба,от 1000001 до 150000
...,...,...,...,...,...,...,...,...,...,...,...,...
21347,1,2194,50,1,1,F,сотрудник,0,145017,свадьба,свадьба,от 1000001 до 150000
21350,1,1330,32,1,1,M,сотрудник,0,240767,сыграть свадьбу,свадьба,от 200000 до 250000
21351,0,9929,57,1,1,M,компаньон,0,157553,свадьба,свадьба,от 150001 до 200000
21357,0,14121,53,1,1,M,пенсионер,0,75439,сыграть свадьбу,свадьба,до 1000000


In [192]:
data_try_pivot = data_try.merge(family_status_dict, on='family_status_id', how='right')
data_try_pivot = data_try_pivot.pivot_table(index='family_status', values='debt', aggfunc=['sum', 'count'])
data_try_pivot

,sum,count
,debt,debt
family_status,,
Не женат / не замужем,0.0,0
в разводе,0.0,0
вдовец / вдова,0.0,0
гражданский брак,183.0,2315
женат / замужем,0.0,0


Кредит на свадьбу берут ***только люди, находящиеся на данный момент в гражданском браке*** и только они. 

### Попробуем отследить статистику людей, находящихся в разводе и имеющих детей

In [201]:
# При помощи словаря вспомним обозначения столбца family_status_id
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [202]:
data_try2=data.loc[data['family_status_id'] == 3]
data_try2=data_try2.pivot_table(index='children', values='debt', aggfunc=['sum', 'count'])
data_try2['children_debt']=data_try2['sum']['debt']/data_try2['count']['debt']
data_try2

,sum,count,children_debt
,debt,debt,
children,,,
0,55,784,0.070153
1,21,316,0.066456
2,7,81,0.086420
3,1,11,0.090909
4,0,1,0.000000


In [203]:
# Для сравнения запустим список, составленный ранее (сравнение задолженности по количеству детей)
data_children_grouped

debt        children_debt
           sum  count              
children                           
0         1063  14090      0.075444
1          445   4855      0.091658
2          194   2052      0.094542
3           27    330      0.081818
4            4     41      0.097561
5            0      9      0.000000

В данном случае можно отследить следующее - чем больше детей у человека, находящегося в разводе - тем ниже показатель возврата кредита в срок. С другой стороны - ***люди, находящиеся в разводе и имеющие 1 ребенка*** отдают кредит вовремя намного чаще (6,6%) чем ***вся категория людей, имеющих одного ребенка***(9,1%). 

### Вывод

Можно отметить влияние целей кредита на его возврат в срок. Люди, берущие кредит на автомобиль или образование имеют больший процент задолженностей (более 9%), в то время как остальные - менее 8%. Следовательно, при цели кредита на автомобиль или образование - риск невозврата кредита вовремя увеличивается. 

### Шаг 4. Общий вывод

***Любые данные***, поступающие на анализ нуждаются в проверке. Почти половина времени работы ушла на доработку исходных данных, т.к. часть данных была пропущена, часть была некорректная, некоторые столбцы пришлось изменять для более удобной и корректной работы. Отвечать на поставленные вопросы без обработки данных будет некорректно.
После корректировки данных - список приобрел более удобный формат, который легче поддавался анализу и минимизировал шанс ошибки. 

В реальных условиях поиск информации мог бы занять другое количество времени, т.к. ***данные следует брать не только из таблицы, но и от тех, кто её составлял либо любым образом владеет корректной информацией***. 

Для удобства при поиске ответов на конкретные вопросы часть таблицы можно урезать, скрывая все лишние данные.
Поэтапный формат работы максимально удобен при анализе, т.к. шанс что-то упустить становится куда меньше, а струкрута позволяет при этом легко вернуться к любому этапу работы при необходимости.
Информация на выходе уже должна быть максимально простой и понятной, чтобы тот человек, который будет обрабатывать результаты мог без труда разобраться.

***Основной вывод*** после проделанной работы - **аналитика такого формата это не только подсчет данных, а куда больший процесс, включающий в себя обработку данных, поиск недостающих и корректировку имеющихся. А значит работать будем не только с техникой, но и с людьми.**




#### Данные анализировались по 4 показателям (цель кредита, уровень дохода, семейное положение и наличие детей).
В каждой из групп мы определяли процентное соотношение задолженности, в среднем показатели составляли ***от 6,5% до 10%***, на основании этого мы и выделяли группы риска и надежные группы.

Деление происходило по следующим показателям:

    -по цели кредита : в группы риска попадают кредиты на автомобиль и образование (более 9% задолженностей), самой надежной группой являлись кредиты на жилье (чуть меньше 7%). 
    
    -по уровню дохода: в зону риска попадают те группы людей, которые получают от 100000 до 150000 и от 150000 до 200000 (около 8,5% и 9% задолженностей соотвественно), люди с большим доходом справляются куда лучше (всего около 7% задолженностей). 
    
    -по семейному положению: лучше всего с кредитами справляются вдовцы/вдовы (всего 6,5%, самая надежная группа из всех), в зоне риска те, кто либо не были в браке, либо находятся в гражданском браке (задолженности превышают 9%).  Женатые и находящиеся в разводе в целом справляются очень даже неплохо (7% и 7,5%). Следует так же отметить тот факт, что женатые составляют больше половины всех клиентов и являются самой многочисленной группой.
    
    -по количеству детей: самая малочисленная группа с семьями из 5 детей (всего 9 клиентов банка) вообще не имеют задолженностей, так же хорошо оплачивают кредиты люди не имеющие детей вообще (7,5% задолженности). В зоне риска люди с 1, 2 и 4 детьми (9,2%, 9,4%, 9,7%), показатели задолженности у которых при сравнении с другими самые высокие!
    
Самым подходящим вариантов для банка будет ***вдовец/вдова, не имеющие детей, с уровнем дохода более 200000 и с целью кредита - жилье***. Больший риск будет исходить от ***тех, кто не был в браке, либо находящимся в гражданском, имеющим 4, 2 или 1 ребенка, с уровнем дохода от 100000 до 200000 и берущем кредит на автомобиль или образование***.
    
При необходимости предоставить анализ в статистических данных вывод будет следующим:

По цели кредита:
    - автомобиль ( всего - 4290, задолженность - 401, процент задолженности - 9,3%)
    - образование (всего - 3998, задолженность - 369, процент задолженности - 9,2%)
    - свадьба (всего - 2315, задолженность - 183, процент задолженности - 7,9%)    
    - недвижимость (всего - 6329, задолженность - 472, процент задолженности - 7,45%)
    - жилье (всего - 4445, задолженность - 308, процент задолженности - 6,9%)

По уровню дохода:
    - до 100000 (всего - 2801, задолженность - 354, процент задолженности - 7,9%)
    - от 100001 до 150000 (всего - 7772, задолженность - 657, процент задолженности - 8,4%)
    - от 150001 до 200000 (всего - 4108, задолженность - 366, процент задолженности - 8,9%)
    - от 200001 до 250000 (всего - 2244, задолженность - 162, процент задолженности - 7,2%)
    - от 250000 (всего - 2801, задолженность - 194, процент задолженности - 6,9%)
    
По семейному положению:
    - не женат / не замужем (всего - 2801, задолженность - 273, процент задолженности - 9,7%)
    - гражданский брак (всего - 4138, задолженность - 385, процент задолженности - 9,3%)
    - женат / замужем (всего - 12990, задолженность - 938, процент задолженности - 7,5%)
    - в разводе (всего - 1193, задолженность - 84, процент задолженности - 7%)
    - вдовец / вдова (всего - 955, задолженность - 63, процент задолженности - 6,5%)
    
По количеству детей:
    - бездетные (всего - 14090, задолженность - 1063, процент задолженности - 7,5%)
    - 1 ребенок (всего - 4855, задолженность - 445, процент задолженности - 9,1%)
    - 2 ребенка (всего - 2052, задолженность - 194, процент задолженности - 9,4%)
    - 3 ребенка (всего - 330, задолженность - 27, процент задолженности - 8,1%)
    - 4 ребенка (всего - 41, задолженность - 4, процент задолженности - 9,7%)
    - 5 ребенков (всего 9, задолженность - 0, процент задолженности - 0%)

    
